In [1]:
!pip install openai

In [4]:
import joblib
from openai import OpenAI
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the churn predictions model
churn_model = joblib.load("./../models/model.joblib")

# Function to generate personalized retention incentives
def generate_retention_incentives(customer_data):
    # Make churn prediction using the churn model
    churn_probability = churn_model.predict_proba(customer_data)[0][1]

    client = OpenAI(
      api_key="YOUR_KEY",
    )
    
    # Generate personalized retention incentive using OpenAI GPT-3.5 Turbo
    prompt = f"Customer churn probability: {churn_probability}\nCustomer data: {customer_data}\nGenerate personalized retention incentive:"
    # completion = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=1000) - Depricated > openai.Completion.create
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
    messages=[       
       {"role": "user", "content": prompt}      
     ]
    )

    retention_incentive = response.choices[0].message.content

    return retention_incentive

data_Unnamed0column = pd.read_csv("./../data/2024-03-20T05-05_export.csv", index_col=[0])
customer_data = data_Unnamed0column

#Defining the map function
def binary_map(feature):
    return feature.map({'Yes':1, 'No':0})

# Encode binary categorical features
binary_list = ['SeniorCitizen','Dependents', 'PhoneService', 'PaperlessBilling']
customer_data[binary_list] = customer_data[binary_list].apply(binary_map)
#feature scaling
#sc = MinMaxScaler()
#customer_data['tenure'] = sc.fit_transform(customer_data[['tenure']])
#customer_data['MonthlyCharges'] = sc.fit_transform(customer_data[['MonthlyCharges']])
#customer_data['TotalCharges'] = sc.fit_transform(customer_data[['TotalCharges']])

columns = ['gender', 'SeniorCitizen', 'Dependents', 'tenure', 'PhoneService', 'PaperlessBilling',
           'MonthlyCharges', 'TotalCharges', 'MultipleLines_No_phone_service', 'MultipleLines_Yes',
           'InternetService_Fiber_optic', 'InternetService_No', 'OnlineSecurity_No_internet_service',
           'OnlineSecurity_Yes', 'OnlineBackup_No_internet_service', 'OnlineBackup_Yes',
           'DeviceProtection_No_internet_service', 'DeviceProtection_Yes', 'TechSupport_No_internet_service',
           'TechSupport_Yes', 'StreamingTV_No_internet_service', 'StreamingTV_Yes', 'StreamingMovies_No_internet_service',
           'StreamingMovies_Yes', 'Contract_One_year', 'Contract_Two_year', 'PaymentMethod_Credit_card__automatic_',
           'PaymentMethod_Electronic_check', 'PaymentMethod_Mailed_check']
#Encoding the other categorical categoric features with more than two categories
customer_data = pd.get_dummies(customer_data).reindex(columns=columns, fill_value=0)

incentive = generate_retention_incentives(customer_data)
print("Retention Incentive:")
print(incentive)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}